In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import kgbench as kg
import fire, sys
import math

from kgbench import load, tic, toc, d


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors


#
from torch_geometric.utils import to_networkx
import networkx as nx

from src.rgcn_explainer_utils import *
from IPython.core.interactiveshell import InteractiveShell
from rgcn_model import RGCN



In [9]:
import os

# Get the current directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Check if the current directory is already the parent directory
if current_dir != '/Users/macoftraopia/Documents/GitHub/RGCN-Explainer':
    # Set the parent directory as the current directory
    os.chdir(parent_dir)
!pwd

/Users/macoftraopia/Documents/GitHub/RGCN-Explainer


In [ ]:

# def convert_binary(sparse_tensor, threshold):
#     # convert values to either 0 or 1 based on a threshold of 0.5
#     mask = sparse_tensor._values() > threshold
#     converted_values = torch.zeros_like(sparse_tensor._values())
#     converted_values[mask] = 1
#     #print number of non zero values
#     print("Number of non zero values: ", converted_values.nonzero().size(0))

#     # create a new sparse tensor with the converted values
#     converted_sparse_tensor = torch.sparse_coo_tensor(sparse_tensor._indices(), converted_values, size=sparse_tensor.size())
#     return converted_sparse_tensor

In [ ]:
name = 'am'
data = kg.load(name, torch=True)
data = prunee(data, 2)
data.withheld #n is num entities
# model = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/model_{name}_prune_True')

In [ ]:
h, v = hor_ver_graph(data.triples, data.num_entities, data.num_relations)

In [ ]:
tuple(h.shape)

In [ ]:
n = data.num_entities
r = data.num_relations
hor_ind, hor_size = adj(data.triples, n, 2*r+1, vertical=False)
ver_ind, ver_size = adj(data.triples, n, 2*r+1, vertical=True)
#number of relations is 2*r+1 because we added the inverse and self loop

_, rn = hor_size #horizontally stacked adjacency matrix size
r = rn // n #number of relations enriched divided by number of nodes

In [ ]:
hor_size

In [ ]:
data.num_relations*2+1
data.num_entities_new

In [ ]:
model = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/model_{name}_prune_True')
out = model.forward2(h,v)
out

In [10]:
name = 'aifb'

data = kg.load(name, torch=True) 
print(data.triples.shape[0])
node_idx = 5678

# else:
#     data = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/data/IMDB/finals/{name}.pt')

data = prunee(data, 2)
print(data.triples.shape[0])
data.triples = torch.Tensor(data.triples).to(int)#data.triples.clone().detach()
data.withheld = torch.Tensor(data.withheld).to(int)#data.withheld.clone().detach()
data.training = torch.Tensor(data.training).to(int)#data.training.clone().detach()
#
get_relations(data)
d_classes(data)
dict_classes = {key.item(): data.withheld[:, 0][data.withheld[:, 1] == key].tolist() for key in torch.unique(data.withheld[:, 1])}
if name != 'aifb':
    node_idx = dict_classes[0][0]


from src.rgcn_explainer_utils import *
# v = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_ver{node_idx}')
# h = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_hor{node_idx}')
model = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/model_{name}_prune_True')
v = torch.load(f'/Users/macoftraopia/Desktop/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_ver{node_idx}')
h = torch.load(f'/Users/macoftraopia/Desktop/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_hor{node_idx}')

# v = torch.sparse_coo_tensor(v.coalesce().indices(), torch.sigmoid(v.coalesce().values()), v.size(), requires_grad=True)
# h = torch.sparse_coo_tensor(h.coalesce().indices(), torch.sigmoid(h.coalesce().values()), h.size(), requires_grad=True)
out = model.forward2(h,v)

res = nn.Softmax(dim=0)(out[node_idx])
print('ypred explain', res)
# print(v.coalesce().values()[v.coalesce().values()>0.5])
# print(h.coalesce().values())

v_bin,h_bin = convert_binary(v, 0.5), convert_binary(h,0.5)
print('v binary:',v_bin, torch.count_nonzero(v_bin.coalesce().values()))
res = nn.Softmax(dim=0)(model.forward2(h_bin,v_bin)[node_idx, :])
print('ypred explain binary', res)

if node_idx in data.withheld[:,0]:
    print('ypred true', data.withheld[data.withheld[:,0]==node_idx,1])
    
from r_exp import hor_ver_graph
model.eval()
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
y_full = model.forward2(hor_graph, ver_graph)
node_pred_full = y_full[node_idx, :]
res_full = nn.Softmax(dim=0)(node_pred_full)
print('ypred full', res_full)

v,h = sub(v, 0.5), sub(h,0.5)
m = match_to_triples(v,h,data, node_idx)
Counter(m[:,1].tolist())

loaded data aifb (0.44s).
29043
26666
ypred explain tensor([0.4521, 0.1908, 0.2938, 0.0633], grad_fn=<SoftmaxBackward0>)
Number of non zero values:  311
Number of non zero values:  311
v binary: tensor(indices=tensor([[ 23451,  23451,  23451,  ..., 329925, 329926, 329927],
                       [  5678,   5743,   5746,  ...,   5230,   5230,   5230]]),
       values=tensor([1., 1., 0.,  ..., 0., 0., 0.]),
       size=(753935, 8285), nnz=1311, layout=torch.sparse_coo) tensor(311)
ypred explain binary tensor([0.4549, 0.1793, 0.2861, 0.0797], grad_fn=<SoftmaxBackward0>)
ypred true tensor([0])
ypred full tensor([9.8647e-01, 5.4700e-04, 1.2982e-02, 1.1929e-06],
       grad_fn=<SoftmaxBackward0>)


Counter({2: 15, 18: 12, 21: 5, 23: 1, 30: 17, 36: 1, 39: 260, 0: 311})

In [7]:
data.i2rel[10]

['fax', 'http://swrc.ontoware.org/ontology#fax']

In [12]:
v_0 = select_on_relation_sparse(v_bin,data, 30)
h_0 = select_on_relation_sparse(h_bin,data, 30)

print(v_0.coalesce().values().count_nonzero(),h_0.coalesce().values().count_nonzero())
out = model.forward2(h_0,v_0)
res = nn.Softmax(dim=0)(out[node_idx])
res

tensor(353) tensor(344)


tensor([0.2741, 0.2371, 0.2457, 0.2432], grad_fn=<SoftmaxBackward0>)

tensor(indices=tensor([[ 23430,  23430,  23430,  ..., 329925, 329926, 329927],
                       [  5757,   5789,   5791,  ...,   5230,   5230,   5230]]),
       values=tensor([0.4677, 0.4697, 0.5104,  ..., 0.4695, 0.4997, 0.4753]),
       size=(753935, 8285), nnz=1132, layout=torch.sparse_coo)

In [4]:
data.triples.shape

torch.Size([26666, 3])

In [ ]:
def select_on_relation_sparse(sparse_tensor,data, relation):
    ''' Selects the values of a sparse tensor based on the relation'''
    output_indices, output_values, value_indices=select_relation(sparse_tensor,data.num_entities,relation)
    coalesced_tensor = sparse_tensor.coalesce()
    coalesced_values = coalesced_tensor._values()
    coalesced_indices = coalesced_tensor._indices()
    coalesced_values[value_indices] = 0
    masked_sparse_tensor = torch.sparse_coo_tensor(coalesced_indices, coalesced_values, sparse_tensor.size())
    return masked_sparse_tensor
  

In [ ]:
def select_one_relation(sparse_tensor,data, relation):
    ''' Selects the values of a sparse tensor based on the relation'''
    sparse_tensor = torch.sparse_coo_tensor(sparse_tensor._indices(), torch.zeros(sparse_tensor._indices().shape[1]), sparse_tensor.size() )
    output_indices, output_values, value_indices=select_relation(sparse_tensor,data.num_entities,relation)
    coalesced_tensor = sparse_tensor.coalesce()
    coalesced_values = coalesced_tensor._values()
    coalesced_indices = coalesced_tensor._indices()
    coalesced_values[value_indices] = 1
    masked_sparse_tensor = torch.sparse_coo_tensor(coalesced_indices, coalesced_values, sparse_tensor.size())
    return masked_sparse_tensor

v = select_one_relation(v,data, 39)
h = select_one_relation(h,data, 39)


In [ ]:
h.coalesce().values().count_nonzero()

In [ ]:
v = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_ver{node_idx}')
h = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/size_0.005_lr_0.1_epochs_30_threshold_0.5_init_normal/masked_adj/masked_hor{node_idx}')
#loop over keys of counter
for key in Counter(m[:,1].tolist()).keys():
    # v_ = select_on_relation_sparse(v,data, key)
    # h_ = select_on_relation_sparse(h,data, key)
    v_ = select_one_relation(v,data, key)
    h_ = select_one_relation(h,data, key)
    out = model.forward2(h_,v_)
    res = nn.Softmax(dim=0)(out[node_idx])
    #print(f'ypred explain no {data.i2r[key]}, {key}', res)
    if torch.argmax(res)!=torch.argmax(res_full):
        pass
        #print(f'wrong prediction without {data.i2r[key]}')
    else:
        print(f'correct only with {data.i2r[key]}')
        print(f'ypred only with {data.i2r[key]}, {key}', res)



In [ ]:
#define a baseline: no use RGCNExplainer - just rule out relations based on prediction of the model

#node:
node_idx = 5678

#label for that node
if node_idx in data.withheld[:,0]:
    print('ypred true', data.withheld[data.withheld[:,0]==node_idx,1])

#edge index
edge_index = edge_index_oneadj(data.triples)

#number of hops
n_hops = 2

#augment dataset with self loops and inverse relations

hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)

#get the edge index at 2 hops per node
_,_,index_h = find_n_hop_neighbors(hor_graph.coalesce().indices(), n_hops, node_idx)
_,_,index_v = find_n_hop_neighbors(ver_graph.coalesce().indices(), n_hops, node_idx)

h = torch.sparse_coo_tensor(index_h, torch.ones(index_h.shape[1]), hor_graph.size() )
v = torch.sparse_coo_tensor(index_v, torch.ones(index_v.shape[1]), ver_graph.size() )

#match to triple
m = match_to_triples(v,h, data)

#counter of relations in the 2 hops subgraph
Counter(m[:,1].tolist())

#forward pass of the model
out = model.forward2(h,v)
res = nn.Softmax(dim=0)(out[node_idx])
print(f'ypred explain all subgraph: {torch.argmax(res)} with prediction probability: {res}')


In [ ]:
Counter(m[:,1].tolist())

In [ ]:
Counter(m[:,1].tolist())

v = select_one_relation(v,data, 39)
h = select_one_relation(h,data, 9)
h.coalesce().values().count_nonzero()

In [ ]:
node_idx = 5757
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
y_full = model.forward2(hor_graph, ver_graph)
node_pred_full = y_full[node_idx, :]
res_full = nn.Softmax(dim=0)(node_pred_full)
print('ypred full', res_full)

m = match_to_triples(ver_graph,hor_graph,data, node_idx)
v, h = ver_graph, hor_graph
for key in Counter(m[:,1].tolist()).keys():
    v_ = select_on_relation_sparse(v,data, key)
    h_ = select_on_relation_sparse(h,data, key)
    out = model.forward2(h_,v_)
    res = nn.Softmax(dim=0)(out[node_idx])
    #print(f'ypred explain no {data.i2r[key]}, {key}', res)
    if torch.argmax(res)!=torch.argmax(res_full):
        print(f'for node {node_idx}, wrong prediction without {data.i2r[key]}')

In [ ]:
for key in Counter(m[:,1].tolist()).keys():
    v_ = select_on_relation_sparse(v,data, key)
    h_ = select_on_relation_sparse(h,data, key)
    out = model.forward2(h_,v_)
    res = nn.Softmax(dim=0)(out[node_idx])
    print(f'ypred explain no {data.i2r[key]}, {key}', res)
    if torch.argmax(res)!=torch.argmax(res_full):
        print(f'wrong prediction without {data.i2r[key]}')

In [ ]:
#get the node of the most important relations - the relations with the highest weights

tensor_list = (list(v.coalesce().indices()[1][v.coalesce().values()>0.5]) + list(h.coalesce().indices()[0][h.coalesce().values()>0.5]))
float_list = [tensor.item() for tensor in tensor_list]
len(set(float_list))


In [ ]:
count = 0
others = []
for i in data.triples:

    if i[0] == 5857:
        print(i)
        count += 1
        others.append(i[2])
    if i[2] == 5857:
        print(i)
        count += 1
        others.append(i[0])

count
others

In [ ]:
count = 0
l = []
a = []
for i in data.triples:
    for n in others:
        if i[0] == int(n) or i[2] == int(n):
            count += 1
            l.append(n)
            a.append(i)
            #print(i)
            break
res = set(l)
resa = set(a)
print(len(resa))            
print(len(res))
count
#others

In [ ]:
def find_repeating_sublists(sublists):
    repeating_elements = {}

    for sublist in sublists:
        key1 = (sublist[0], sublist[2])
        key2 = (sublist[2], sublist[0])

        if key1 in repeating_elements:
            repeating_elements[key1].append(sublist[1])
        elif key2 in repeating_elements:
            repeating_elements[key2].append(sublist[1])
        else:
            repeating_elements[key1] = [sublist[1]]

    result_array = []
    for key, values in repeating_elements.items():
        if len(values) > 1:
            result_array.append([key[0], values, key[1]])
        else:
            result_array.append([key[0], [values[0]], key[1]])

    return result_array

def unnest_list(nested_list):
    return [item for sublist in nested_list for item in (unnest_list(sublist) if isinstance(sublist, list) else [sublist])]

def visualize(node_idx, n_hop, data, masked_ver,threshold,name, result_weights=True, low_threshold=False ):
    """ 
    Visualize important nodes for node idx prediction
    """
    dict_index = dict_index_classes(data,masked_ver)
    
    #select only nodes with a certain threshold
    sel_masked_ver = sub_sparse_tensor(masked_ver, threshold,data, low_threshold)
    if len(sel_masked_ver)==0:
        sel_masked_ver=sub_sparse_tensor(masked_ver, 0,data, low_threshold)
    print('sel masked ver',sel_masked_ver)
    indices_nodes = sel_masked_ver.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) #original edge indexes

    
    
    G = nx.Graph()
    if result_weights:
        values = sel_masked_ver.coalesce().values().detach().numpy()
        for s,p,o in zip(indices_nodes[0],values , indices_nodes[1]):
            G.add_edge(int(s), int(o), weight=np.round(p, 2))

    else:

        triples_matched = match_to_triples(sel_masked_ver, data)
        l = []
        for i in triples_matched[:,1]:
            l.append(data.i2rel[int(i)][0])
        triples_matched = find_repeating_sublists(triples_matched.numpy())
        for s,p,o in triples_matched:
            G.add_edge(int(s), int(o), weight=p)


    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    weights = [[item] if not isinstance(item, list) else item for item in weights]


    pos = nx.circular_layout(G)

    ordered_dict = {}
    for item in list(G.nodes):
        if item in ordered_dict:
            ordered_dict[item].append(dict_index[item])
        else:
            ordered_dict[item] =  dict_index[item]

    dict_index = ordered_dict

    labeldict = {}
    for node in G.nodes:
        labeldict[int(node)] = int(node)  


    dict = {}
    for k,v in dict_index.items():
        for k1,v1 in data.entities_classes.items():
            if v==k1: 

                dict[k] = v1
            else:
                if k not in dict:
                    dict[k] = 0
                

    color_list = list(dict.values())
    color_list = list(encode_dict(dict_index).values())


    col_weights = [weights[i][0] for i in range(len(weights))]
    if result_weights:
        
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=col_weights, node_color =  color_list, cmap="Set2",edge_cmap=plt.cm.Reds,font_size=8)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=nx.get_edge_attributes(G,'weight'),font_size=8,font_color='red')
        sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=0, vmax=1))
        sm.set_array(weights)
        cbar = plt.colorbar(sm)
        cbar.ax.set_title('Weight')
        plt.title("Node {}'s {}-hop neighborhood important nodes".format(node_idx, n_hop))
    else:
        rel = nx.get_edge_attributes(G,'weight')
        rel = {k: [data.i2rel[i][0] for i in v] for k,v in rel.items()}
        col_weights = [sum(weights[i], 3) if len(weights[i]) > 1 else weights[i][0] for i in range(len(weights))]
        nx.draw(G, pos,labels = labeldict, edge_color=col_weights,edgelist=edges,node_color =  color_list, cmap="Set2",font_size=7, arrows = True)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=rel,font_size=8,font_color='red')
        
        res = Counter(unnest_list(rel.values()))
    if result_weights:
        if not os.path.exists(f'chk/{name}_chk/graphs'):
            os.makedirs(f'chk/{name}_chk/graphs')  
        plt.savefig(f'chk/{name}_chk/graphs/Explanation_{node_idx}_{n_hop}_weights.png')

        #plt.show()

    else:
        if not os.path.exists(f'chk/{name}_chk/graphs'):
            os.makedirs(f'chk/{name}_chk/graphs')  
        plt.savefig(f'chk/{name}_chk/graphs/Explanation_{node_idx}_{n_hop}_relations.png')    
        #plt.show()
        return res, weights
    

In [ ]:

v = torch.load(f'/Users/macoftraopia/Documents/GitHub/RGCN-Explainer/chk/{name}_chk/masked_adj/masked_ver{node_idx}_new')
res, weights = visualize(node_idx, 2, data, v, 0, name, result_weights=False, low_threshold=False)

In [ ]:
import torch

def hor_ver_graph(triples, n, r):
    """
    input: triples, number of nodes, number of relations
    output: hor_graph, ver_graph : horizontally and vertically stacked adjacency matrix
    """

    hor_ind, hor_size = adj(triples, n, 2*r+1, vertical=False)
    ver_ind, ver_size = adj(triples, n, 2*r+1, vertical=True)

    rn, _ = hor_size  # horizontally stacked adjacency matrix size
    r = rn // n  # number of relations enriched divided by number of nodes

    vals = torch.ones(ver_ind.size(0), dtype=torch.float)  # number of enriched triples

    hor_graph = torch.sparse.FloatTensor(hor_ind.t(), vals, hor_size)  # size: n, r, emb
    ver_graph = torch.sparse.FloatTensor(ver_ind.t(), vals, ver_size)

    return hor_graph, ver_graph

def adj(triples, num_nodes, num_rels, cuda=False, vertical=True):
    """
     Computes a sparse adjacency matrix for the given graph (the adjacency matrices of all
     relations are stacked vertically).

     :param edges: List representing the triples
     :param i2r: list of relations
     :param i2n: list of nodes
     :return: sparse tensor
    """
    r, n = num_rels, num_nodes
    size = (r * n, n) if vertical else (n, r * n)

    from_indices = []
    upto_indices = []

    for s, p, o in triples:
        offset = p.item() * n
        s = offset + s.item() if vertical else s.item()
        o = offset + o.item() if not vertical else o.item()
        from_indices.append(s)
        upto_indices.append(o)

    indices = torch.tensor([from_indices, upto_indices], dtype=torch.long, device=d(cuda))

    return indices.t(), size



In [ ]:
adj(data.triples, data.num_entities, data.num_relations, cuda=False, vertical=True)
hor_ver_graph(data.triples, data.num_entities, data.num_relations)

In [ ]:

hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
edge_h, edge_v = hor_graph.coalesce().indices(), ver_graph.coalesce().indices()
_,_,sub_edges_tensor_h  = find_n_hop_neighbors(edge_h,2, 5699)
_,_,sub_edges_tensor_v  = find_n_hop_neighbors(edge_v,2, 5699)


In [ ]:
len(sub_edges_tensor_v)

In [ ]:
sub_edges_tensor
indexes = sub_edges_tensor%data.num_entities
indexes
r = sub_edges_tensor//data.num_entities
r

In [ ]:
_,p = torch.div(sub_edges_tensor, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
s,o = sub_edges_tensor%data.num_entities
result = torch.stack([s,p,o], dim=1)
result

In [ ]:
#try initialization where the first hop neighborhood gets initialized with 1s and the rest with different methods??
def construct_edge_mask(self, num_nodes,sparse_tensor,data, const_val=1.0, relation_id = 2):
    """
    Construct edge mask
    """
    init_strategy = self.init_strategy
    # if num_nodes > 1000:
    #     init(strategy="const", const_val=0.1)
    data = self.data
    num_entities = data.num_entities
    torch.manual_seed(42)
    mask = nn.Parameter(torch.FloatTensor(num_nodes))

    if init_strategy == "normal":
        std = nn.init.calculate_gain("relu") * math.sqrt(
            2.0 / (num_nodes + num_nodes)
        )
        with torch.no_grad():
            mask.normal_(1.0, std)
    elif init_strategy == "const":
        nn.init.constant_(mask, const_val) 
    elif init_strategy == "zero_out":
        '''initialize the mask with the zero out strategy: we zero out edges belonging to specific relations'''
        std = nn.init.calculate_gain("relu") * math.sqrt(
            2.0 / (num_nodes + num_nodes)
        )
        with torch.no_grad():
            mask.normal_(1.0, std)
        output_indices, output_values, value_indices=select_relation(sparse_tensor,relation_id)
        _,_,value_indices1=select_relation(sparse_tensor,33)
        print(value_indices, value_indices1)
        value_indices = torch.cat((value_indices, value_indices1), 0)
        mask.data[[value_indices]] = 0
    

    elif init_strategy == "overall_frequency":
        '''Initialize the mask with the overall frequency of the relations'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        overall_rel_frequency = dict(Counter(data.triples[:,1].tolist()))#.most_common()

        overall_rel_frequency_  = {key: round(value/len(data.triples[:,1].tolist()),5) for key, value in overall_rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = overall_rel_frequency_[i]
    
    elif init_strategy == "relative_frequency":
        ''' Initialize the mask with the relative frequency of the relations-relative for the node to be explained'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        rel_frequency = dict(Counter(p))
        rel_frequency_  = {key: round(value/len(p),5) for key, value in rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = rel_frequency_[i]

    elif init_strategy == "inverse_relative_frequency":
        ''' Initialize the mask with the relative frequency of the relations-relative for the node to be explained'''
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        rel_frequency = dict(Counter(p))
        rel_frequency_  = {key: 1 - round(value/len(p),5) for key, value in rel_frequency.items()}
        for i in p:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = rel_frequency_[i]


    elif init_strategy == "domain_frequency":
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        dict_domain, dict_range = domain_range_freq(data, len(d_classes(data)))
        for i in p:

            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = dict_domain[i]

    elif init_strategy == "range_frequency":
        _ ,p = torch.div(sparse_tensor.coalesce().indices(), num_entities, rounding_mode='floor').tolist()
        dict_domain, dict_range = domain_range_freq(data, len(d_classes(data)))
        for i in p:
                _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
                mask.data[[value_indices]] = dict_range[i]
    elif init_strategy == "rdf":
        rdf = [i for i in range(data.num_relations) if 'rdf' in data.i2r[i]]
        for i in rdf:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = 0
    elif init_strategy == "owl":
        owl = [i for i in range(data.num_relations) if 'owl' in data.i2r[i]]
        for i in owl:
            _,_,value_indices=select_relation(sparse_tensor,num_entities,i)
            mask.data[[value_indices]] = 0
    print(f'mask initialized with {init_strategy} strategy: {mask}')   
    return mask

In [ ]:
import numpy as np
np.exp(-10)

In [ ]:

v  = torch.load('chk/aifb_chk/size_0.05_lr_0.1_epochs_30_threshold_0.7_init_normal/masked_adj/masked_ver5757')
h = torch.load('chk/aifb_chk/size_0.05_lr_0.1_epochs_30_threshold_0.7_init_normal/masked_adj/masked_hor5757')

In [ ]:
v

In [ ]:
nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() > 0.7]
nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
nonzero_values = v.coalesce().values()[v.coalesce().values() > 0.7]
sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
sel_masked_ver

In [ ]:
v = sub_sparse_tensor(v, 0.5, data, low_threshold=False)
h = sub_sparse_tensor(h, 0.5, data, low_threshold=False)

In [ ]:
def sub(v):
    nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() < 0.5]
    nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
    nonzero_values = v.coalesce().values()[v.coalesce().values() < 0.5]
    sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    return sel_masked_ver
    

In [ ]:
def sub(v, threshold):
    nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() > threshold]
    nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
    nonzero_values = v.coalesce().values()[v.coalesce().values() > threshold]
    sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    return sel_masked_ver
sub(v, 0.5)

In [ ]:
def match_to_triples(v,h, data, sparse=True):
    if sparse:
        pv,_ = torch.div(v.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sv,ov = v.coalesce().indices()%data.num_entities
        result_v = torch.stack([sv,pv,ov], dim=1)
        ph,_ = torch.div(h.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sh,oh = h.coalesce().indices()%data.num_entities
        result_h = torch.stack([sh,ph,oh], dim=1)
        result = torch.cat((result_v, result_h), 0)


                    
    else:
        if len(h )!= 0:
            _,ph = torch.div(h, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sh,oh = h%data.num_entities
            result_h = torch.stack([sh,ph,oh], dim=1)
        if len(v)!=0:
            pv, _ = torch.div(v, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sv,ov = v%data.num_entities
            result_v = torch.stack([sv,pv,ov], dim=1)
        if len(h) != 0 and len(v) != 0:
            result = torch.cat((result_v, result_h), 0)
            print(pv,ph)
        if len(h) == 0:
            result = result_v
            print(pv)
        if len(v) == 0:
            result = result_h
            print(ph)
        

                    
    
    return result

m = match_to_triples(v,h, data, sparse=True)
Counter(m[:,1].tolist())




In [ ]:
def match_to_triples(v, data, sparse=True):
    if sparse:
        # p,_ = torch.div(v.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        # s,o = v.coalesce().indices()%data.num_entities
        # result = torch.stack([s,p,o], dim=1)
        matching = []
        indexes = v.coalesce().indices()%data.num_entities
        for j in range(indexes.size()[1]):
            for triple in data.triples:
                if triple[0] == indexes[0][j] and triple[2] == indexes[1][j]:
                    matching.append(triple)
        result = torch.stack(matching)

                    
    else:
        matching = []
        for i,i2 in zip(v[:,0],v[:,1]):
            for j,j1,j2, index in zip(data[:,0],data[:,1],  data[:,2], range(len(data[:,0]))):
                if i == j and i2 == j2:
                    matching.append(data[index])
                    

        result = torch.stack(matching)
    
    return result
m = match_to_triples(h, data, sparse=True)
Counter(m[:,1].tolist())

In [ ]:
def sub(v, threshold):
    nonzero_indices = v.coalesce().indices()[:, v.coalesce().values() > threshold]
    nonzero_indices[0] = nonzero_indices[0]#%data.num_entities
    nonzero_values = v.coalesce().values()[v.coalesce().values() > threshold]
    sel_masked_ver = torch.sparse_coo_tensor(nonzero_indices, nonzero_values)
    return sel_masked_ver

def visualize(node_idx, n_hop, data, masked_ver,masked_hor, threshold,name, result_weights=True, low_threshold=False,experiment_name=None ):
    """ 
    Visualize important nodes for node idx prediction
    """
    dict_index = dict_index_classes(data,masked_ver)
    mask = torch.vstack((masked_ver, masked_hor.t()))
    mask = sub(mask, threshold)
    print(mask)
    #select only nodes with a certain threshold
    sel_masked_ver = sub(masked_ver, threshold)
    sel_masked_hor = sub(masked_hor, threshold)
    if len(sel_masked_ver)==0:
        sel_masked_ver=sub_sparse_tensor(masked_ver, 0,data, low_threshold)
    #mask = torch.vstack((sel_masked_ver, sel_masked_hor.t()))
    print('sel masked ver',mask)
    indices_nodes = mask.coalesce().indices().detach().numpy()
    new_index = np.transpose(np.stack((indices_nodes[0], indices_nodes[1]))) #original edge indexes

    
    
    G = nx.Graph()
    if result_weights:
        values = mask.coalesce().values().detach().numpy()
        for s,p,o in zip(indices_nodes[0],values , indices_nodes[1]):
            G.add_edge(int(s), int(o), weight=np.round(p, 2))

    else:

        triples_matched = match_to_triples(sel_masked_ver,sel_masked_hor, data)
        l = []
        for i in triples_matched[:,1]:
            l.append(data.i2rel[int(i)][0])
        triples_matched = find_repeating_sublists(triples_matched.numpy())
        print(triples_matched)
        for s,p,o in triples_matched:
            G.add_edge(int(s), int(o), weight=p)


    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    weights = [[item] if not isinstance(item, list) else item for item in weights]


    pos = nx.circular_layout(G)

    ordered_dict = {}
    for item in list(G.nodes):
        if item in ordered_dict:
            ordered_dict[item].append(dict_index[item])
        # else:
        #     ordered_dict[item] =  dict_index[item]

    dict_index = ordered_dict

    labeldict = {}
    for node in G.nodes:
        labeldict[int(node)] = int(node)  


    dict = {}
    for k,v in dict_index.items():
        for k1,v1 in data.entities_classes.items():
            if v==k1: 

                dict[k] = v1
            else:
                if k not in dict:
                    dict[k] = 0
                

    color_list = list(dict.values())
    color_list = list(encode_dict(dict_index).values())


    col_weights = [weights[i][0] for i in range(len(weights))]
    if result_weights:
        
        nx.draw(G, pos,labels = labeldict,  edgelist=edges, edge_color=col_weights, node_color =  color_list, cmap="Set2",edge_cmap=plt.cm.Reds,font_size=8)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=nx.get_edge_attributes(G,'weight'),font_size=8,font_color='red')
        sm = plt.cm.ScalarMappable(cmap=plt.cm.Reds, norm=plt.Normalize(vmin=0, vmax=1))
        sm.set_array(weights)
        cbar = plt.colorbar(sm)
        cbar.ax.set_title('Weight')
        plt.title("Node {}'s {}-hop neighborhood important nodes".format(node_idx, n_hop))
    else:
        rel = nx.get_edge_attributes(G,'weight')
        rel = {k: [data.i2rel[i][0] for i in v] for k,v in rel.items()}
        col_weights = [sum(weights[i], 3) if len(weights[i]) > 1 else weights[i][0] for i in range(len(weights))]
        nx.draw(G, pos,labels = labeldict, edge_color=col_weights,edgelist=edges,node_color =  color_list, cmap="Set2",font_size=7, arrows = True)
        nx.draw_networkx_edge_labels( G, pos,edge_labels=rel,font_size=8,font_color='red')
        
        res = Counter(unnest_list(rel.values()))
        print(res)
    if result_weights:
        if not os.path.exists(f'chk/{name}_chk/{experiment_name}⁄graphs'):
            os.makedirs(f'chk/{name}_chk/{experiment_name}⁄graphs')  
        plt.savefig(f'chk/{name}_chk/{experiment_name}⁄graphs/Explanation_{node_idx}_weights.png')

        #plt.show()

    else:
        if not os.path.exists(f'chk/{name}_chk/{experiment_name}⁄graphs'):
            os.makedirs(f'chk/{name}_chk/{experiment_name}⁄graphs')  
        plt.savefig(f'chk/{name}_chk/{experiment_name}⁄graphs/Explanation_{node_idx}_relations.png')    
        #plt.show()
        return res, weights
    


In [ ]:
torch.vstack((v,h.t()))

In [ ]:
visualize(node_idx, 2, data, v,h, 0.5,name, result_weights=False, low_threshold=False,experiment_name=None )

In [11]:
def match_to_triples(v,h, data, sparse=True):
    if sparse:
        pv,_ = torch.div(v.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sv,ov = v.coalesce().indices()%data.num_entities
        result_v = torch.stack([sv,pv,ov], dim=1)
        ph,_ = torch.div(h.coalesce().indices(), data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sh,oh = h.coalesce().indices()%data.num_entities
        result_h = torch.stack([sh,ph,oh], dim=1)
        result = torch.cat((result_v, result_h), 0)


                    
    else:

        _,ph = torch.div(h, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sh,oh = h%data.num_entities
        result_h = torch.stack([sh,ph,oh], dim=1)

        pv, _ = torch.div(v, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
        sv,ov = v%data.num_entities
        result_v = torch.stack([sv,pv,ov], dim=1)

        result = torch.cat((result_v, result_h), 0)

        if len(h )!= 0:
            _,ph = torch.div(h, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sh,oh = h%data.num_entities
            result_h = torch.stack([sh,ph,oh], dim=1)
        if len(v)!=0:
            pv, _ = torch.div(v, data.num_entities, rounding_mode='floor')#v.coalesce().indices()//data.num_entities
            sv,ov = v%data.num_entities
            result_v = torch.stack([sv,pv,ov], dim=1)
        if len(h) != 0 and len(v) != 0:
            result = torch.cat((result_v, result_h), 0)
            print(pv,ph)
        if len(h) == 0:
            result = result_v
            print(pv)
        if len(v) == 0:
            result = result_h
            print(ph)
        

                    
    
    return result

In [19]:
hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
n_hops = 0
node_idx = 5678
#hor_graph, ver_graph = hor_ver_graph(data.triples, data.num_entities, data.num_relations)
edge_index_h, edge_index_v = hor_graph.coalesce().indices(), ver_graph.coalesce().indices()
sub_edges, neighbors, sub_edges_tensor_h  = find_n_hop_neighbors(edge_index_h, n=n_hops, node=node_idx)

sub_edges, neighbors, sub_edges_tensor_v  = find_n_hop_neighbors(edge_index_v, n=n_hops, node=node_idx)
print(len(list(neighbors)))
print('shape sub',sub_edges_tensor_h.shape, sub_edges_tensor_v.shape)

26
shape sub torch.Size([2, 25]) torch.Size([2, 26])


In [21]:
node_idx = 5757
count = 0


def neighbors
for m in data.triples:
    if m[0] == node_idx or m[2] == node_idx:
        print(m)
        count += 1
print(count)

tensor([5757,   10,  908])
tensor([5757,   23, 2227])
tensor([5757,   27, 1002])
tensor([5757,   30, 6860])
tensor([5757,   30, 6874])
tensor([5757,   30, 6920])
tensor([5757,   30, 6976])
tensor([5757,   30, 7731])
tensor([5757,   30, 7837])
tensor([5757,   30, 7857])
tensor([5757,   30, 7905])
tensor([5757,   30, 7933])
tensor([5757,   30, 7973])
tensor([5757,   39, 5230])
tensor([6860,    2, 5757])
tensor([6874,    2, 5757])
tensor([6920,    2, 5757])
tensor([6976,    2, 5757])
tensor([7731,    2, 5757])
tensor([7837,    2, 5757])
tensor([7857,    2, 5757])
tensor([7905,    2, 5757])
tensor([7933,    2, 5757])
tensor([7973,    2, 5757])
24
